# Yêu cầu
* TẠO MỚI Lab03.03
* Giới thiệu tổng quan về thuật toán louvain
* Tóm tắt tổng quan các chỉ số đánh giá việc phát hiện cộng đồng (công thức, phạm vi, như thế nào là tốt, .... )
* Thực hiện phân tích việc (nhận xét) phát hiện cộng đồng cho mạng xã hội tự chọn

# Thuật toán louvain

## Mục tiêu
- Thuật toán Louvain tìm cách tối ưu hóa modularity, một thước đo để đánh giá chất lượng của phân hoạch cộng đồng. Modularity phản ánh mức độ các cạnh trong cùng một cộng đồng nhiều hơn so với kỳ vọng ngẫu nhiên. Giá trị modularity nằm trong khoảng từ -1 đến 1, và giá trị càng cao thì cộng đồng càng rõ ràng.




## Nguyên lí hoạt động
 Thuật toán Louvain bao gồm hai giai đoạn chính, lặp lại theo từng cấp độ (level):

1.   Giai đoạn tối ưu hóa cục bộ
- Mỗi đỉnh ban đầu được xem như một cộng đồng riêng lẻ.
- Xem xét lần lượt từng đỉnh và kiểm tra việc di chuyển đỉnh đó sang cộng đồng lân cận nào làm tăng modularity nhiều nhất.
- Nếu không có cải thiện modularity khi di chuyển, đỉnh sẽ giữ nguyên cộng đồng ban đầu.

2.   Giai đoạn xây dựng lại đồ thị
- Sau khi không thể cải thiện modularity trong giai đoạn đầu, mỗi cộng đồng được gom thành một "siêu đỉnh".
- Các cạnh giữa các siêu đỉnh được tính bằng tổng trọng số các cạnh giữa các đỉnh thuộc cộng đồng tương ứng.

Hai giai đoạn này được lặp lại cho đến khi modularity không còn tăng thêm đáng kể, hoặc đạt giá trị tối ưu.



## Ưu điểm
- Hiệu quả cao: Louvain có độ phức tạp thời gian tuyến tính
𝑂
(
𝑛
log
⁡
𝑛
)
O(nlogn), phù hợp để xử lý các đồ thị lớn với hàng triệu đỉnh.
- Phân cấp cộng đồng: Kết quả của thuật toán không chỉ cung cấp một phân hoạch mà còn thể hiện cấu trúc phân cấp của cộng đồng (community hierarchy).
- Khả năng mở rộng: Hoạt động tốt trên đồ thị không trọng số, đồ thị trọng số, và đồ thị định hướng.

## Nhược điểm
- Phụ thuộc vào thứ tự đỉnh: Kết quả có thể thay đổi nếu thứ tự đỉnh được xử lý khác nhau.
- Bẫy cục bộ: Thuật toán có thể dừng lại ở cực trị cục bộ của modularity, dẫn đến việc phát hiện cộng đồng không tối ưu trong một số trường hợp.

# Các chỉ số đánh giá việc phát hiện cộng đồng

In [8]:
import pandas as pd

# Define metrics summary data
metrics_summary = [
    {
        "Metric": "Modularity",
        # "Formula": r"$Q = \frac{1}{2m} \sum_{i,j} \left[ A_{ij} - \frac{k_i k_j}{2m} \right] \delta(c_i, c_j)$",
        "Range": "[-1, 1]",
        "Good When": "Closer to 1"
    },
    {
        "Metric": "Conductance",
        # "Formula": r"$\phi(S) = \frac{\text{Cắt}(S, \bar{S})}{\min(\text{Vol}(S), \text{Vol}(\bar{S}))}$",
        "Range": "[0, 1]",
        "Good When": "Closer to 0"
    },
    {
        "Metric": "Normalized Cut",
        # "Formula": r"$\text{NCut}(S, \bar{S}) = \frac{\text{Cắt}(S, \bar{S})}{\text{Vol}(S)} + \frac{\text{Cắt}(S, \bar{S})}{\text{Vol}(\bar{S})}$",
        "Range": "> 0",
        "Good When": "Closer to 0"
    },
    {
        "Metric": "Density",
        # "Formula": r"$\rho(C) = \frac{2 |E_C|}{|C| (|C| - 1)}$",
        "Range": "[0, 1]",
        "Good When": "Closer to 1"
    },
    {
        "Metric": "Silhouette Score",
        # "Formula": r"$S(i) = \frac{b(i) - a(i)}{\max(a(i), b(i))}$",
        "Range": "[-1, 1]",
        "Good When": "Closer to 1"
    },
    {
        "Metric": "Coverage",
        # "Formula": r"$\text{Coverage} = \frac{\text{Tổng trọng số cạnh nội bộ}}{\text{Tổng trọng số cạnh trong đồ thị}}$",
        "Range": "[0, 1]",
        "Good When": "Closer to 1"
    },
    {
        "Metric": "Adjusted Rand Index (ARI)",
        # "Formula": r"$\text{ARI} = \frac{\text{Index quan sát - Index ngẫu nhiên}}{\text{Index tối đa - Index ngẫu nhiên}}$",
        "Range": "[-1, 1]",
        "Good When": "Closer to 1"
    },
    {
        "Metric": "Purity",
        # "Formula": r"$\text{Purity} = \frac{1}{N} \sum_k \max_j |C_k \cap L_j|$",
        "Range": "[0, 1]",
        "Good When": "Closer to 1"
    },
]

# Create a DataFrame
df_metrics_summary = pd.DataFrame(metrics_summary)

# Display the DataFrame to the user
# import ace_tools as tools; tools.display_dataframe_to_user(name="Community Detection Metrics Summary", dataframe=df_metrics_summary)


In [10]:
from IPython.display import display, HTML

# Render DataFrame with LaTeX-like formulas in HTML
def render_formula_dataframe(df):
    html = "<table style='border-collapse: collapse; width: 100%;'>"
    html += "<tr style='background-color: #f2f2f2;'><th>Metric</th><th>Formula</th><th>Range</th><th>Good When</th></tr>"
    for _, row in df.iterrows():
        html += f"<tr style='border: 1px solid black;'>"
        html += f"<td style='padding: 8px;'>{row['Metric']}</td>"
        # html += f"<td style='padding: 8px;'>{row['Formula']}</td>"
        html += f"<td style='padding: 8px;'>{row['Range']}</td>"
        html += f"<td style='padding: 8px;'>{row['Good When']}</td>"
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

# Render the DataFrame
render_formula_dataframe(df_metrics_summary)


# Ví dụ

In [1]:
import networkx as nx
import numpy as np
from sklearn.metrics import silhouette_score
from networkx.algorithms import community

# Example graph (you can replace this with your own graph)
G = nx.karate_club_graph()

# Example partitions (replace with your detected communities)
# Each community is a set of nodes
partition = [set(c) for c in community.louvain_communities(G)]

# Functions for metrics calculation
def modularity(G, partition):
    return community.modularity(G, partition)

def conductance(G, partition):
    conductances = []
    for comm in partition:
        comm_edges = sum(1 for edge in G.edges(comm) if edge[1] in comm)
        cut_edges = sum(1 for edge in G.edges(comm) if edge[1] not in comm)
        vol_comm = 2 * comm_edges + cut_edges
        conductances.append(cut_edges / vol_comm if vol_comm > 0 else 0)
    return np.mean(conductances)

def normalized_cut(G, partition):
    ncut = 0
    for comm in partition:
        cut_edges = sum(1 for edge in G.edges(comm) if edge[1] not in comm)
        vol_comm = sum(degree for node, degree in G.degree(comm))
        ncut += (cut_edges / vol_comm) if vol_comm > 0 else 0
    return ncut

def density(partition, G):
    densities = []
    for comm in partition:
        subgraph = G.subgraph(comm)
        n = subgraph.number_of_nodes()
        if n > 1:
            densities.append(2 * subgraph.number_of_edges() / (n * (n - 1)))
        else:
            densities.append(0)
    return np.mean(densities)

def silhouette_score_custom(G, partition):
    labels = []
    node_to_community = {}
    for i, comm in enumerate(partition):
        for node in comm:
            node_to_community[node] = i
    labels = [node_to_community[node] for node in G.nodes()]
    adjacency_matrix = nx.to_numpy_array(G)
    return silhouette_score(adjacency_matrix, labels)

# Calculate metrics
metrics = {
    "Modularity": modularity(G, partition),
    "Conductance": conductance(G, partition),
    "Normalized Cut": normalized_cut(G, partition),
    "Density": density(partition, G),
    "Silhouette Score": silhouette_score_custom(G, partition),
}

# Display results
print("Community Detection Metrics Summary:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")


Community Detection Metrics Summary:
Modularity: 0.4345
Conductance: 0.2028
Normalized Cut: 0.6083
Density: 0.4156
Silhouette Score: 0.1061


## Nhận xét
1. Modularity (0.4345):

Modularity thể hiện mức độ các cộng đồng được phân hoạch tốt hơn so với một phân hoạch ngẫu nhiên.
 giá trị này nằm trong khoảng "chất lượng khá tốt", vì giá trị 𝑄> thường được coi là chấp nhận được. Tuy nhiên, vẫn còn không gian để cải thiện chất lượng phân cụm bằng các thuật toán khác hoặc điều chỉnh tham số hiện tại.
2. Conductance (0.2028):
Giá trị này khá thấp, cho thấy các cộng đồng được xác định có tính độc lập cao và ít giao thoa với nhau. Conductance càng gần 0 thì chất lượng cộng đồng càng tốt.
3. Normalized Cut (0.6083):
Giá trị này nằm ở mức trung bình. Normalized Cut càng thấp thì cộng đồng càng ít liên kết với phần còn lại của đồ thị.
Với NCut>0.5, cho thấy ranh giới phân cụm chưa thật sự tối ưu, và có thể cải thiện bằng các phương pháp phân cụm khác hoặc tinh chỉnh tham số.
4. Density (0.4156):
Giá trị mật độ khá cao, thể hiện các cộng đồng có mối liên kết nội bộ tương đối chặt chẽ.
Với ρ≈0.42, các cộng đồng có mức độ liên kết khá tốt.
5. Silhouette Score (0.1061):
Giá trị rất thấp, cho thấy các cộng đồng được phân hoạch không rõ ràng khi xem xét khoảng cách giữa các nút.
Silhouette Score > 0.25 thường biểu thị phân hoạch hợp lý, do đó cần xem xét lại các phương pháp hoặc thuật toán phân hoạch.